In [38]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))

In [147]:
import ee 
import geemap

#ee.Initialize(project="ee-sahellakes")
ee.Initialize(project="ee-india-reservoirs")

In [113]:
from src.data_processing.downscaling import resample_image, Downscaler
from src.data_processing.sentinel_preprocessing import load_sentinel2_data

from utils.date_utils import (
    set_to_first_of_month,
    print_collection_dates,
    create_centered_date_ranges,
)
from utils.ee_utils import harmonized_ts, export_image_to_asset, back_to_int, back_to_float
from utils.harmonic_regressor import HarmonicRegressor

from typing import List, Callable, Tuple

In [117]:
import importlib
from utils import wapor_et_processing
from utils import harmonic_regressor
from utils import ee_utils
from utils import date_utils

# Reload the module to apply changes
importlib.reload(wapor_et_processing)
regressor = HarmonicRegressor()
print(hasattr(regressor, "fit2"))  # Should return True

# Reload the module
importlib.reload(ee_utils)
importlib.reload(date_utils)

# Re-import harmonized_ts to ensure updated code is loaded
from utils.ee_utils import harmonized_ts, MosaicType
from utils.date_utils import create_monthly_date_ranges

# Verify the updated MosaicType
print(MosaicType.__members__)


True
{'RECENT': <MosaicType.RECENT: 'recent'>, 'LEAST_CLOUDY': <MosaicType.LEAST_CLOUDY: 'least_cloudy'>, 'MAX_NDVI': <MosaicType.MAX_NDVI: 'max_NDVI'>}


In [99]:
def cloudscore_L8_T1L2(image):
    """
    Apply cloud masking, scaling, and metadata extraction for Landsat 8 Collection 2 Level 2 imagery.
    
    Args:
        image (ee.Image): Input Landsat 8 image.
    
    Returns:
        ee.Image: Processed image with cloud and mask bands, metadata, and scaling applied.
    """
    # Bitmask for QA_PIXEL
    # Bit 0 - Fill, Bit 1 - Dilated Cloud, Bit 2 - Cirrus, Bit 3 - Cloud, Bit 4 - Cloud Shadow
    qa_mask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    # Mask for valid pixels
    mask = qa_mask

    # Create cloud band (value 100 for all pixels, 0 for cloud-free pixels in qa_mask)
    image2 = ee.Image(100).rename('cloud').where(qa_mask.eq(1), 0)

    # Update mask for QA_PIXEL values not equal to 0
    image3 = image2.updateMask(image.select('QA_PIXEL').neq(0))

    # Scale optical bands (SR_B*) and thermal bands (ST_B*)
    optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0).subtract(273)
    # Compute NDVI (Normalized Difference Vegetation Index)
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

    # Add scaled bands, cloud band, mask, and metadata
    processed_image = (
        image.addBands(optical_bands, None, True)
        .addBands(thermal_bands, None, True).addBands(ndvi)
        .updateMask(image2.select(['cloud']).lt(100))
        .addBands(image2)
        .addBands(mask.rename('mask'))
        .set('SENSING_TIME', ee.Date(image.get('system:time_start')).format('YYYY-MM-dd'))
        .set('SATELLITE', 'LANDSAT_8')
    )

    return processed_image


---

In [160]:

# List of countries
countries = ee.List([
    'Albania',
    'Bulgaria',
    'Bosnia & Herzegovina',
    'Croatia',
    'Moldova',
    'Macedonia',
    'Montenegro',
    'Poland',
    'Romania',
    'Serbia',
    'Turkey',
    'Kosovo'
])

# Load the FeatureCollection
grid_results = ee.FeatureCollection('projects/ee-et-data/assets/ECA/GRID_area_irrg_WorldCereal_Corine_v2')
borders=ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
# Map over the FeatureCollection to add a 'combined' property
grid_results = grid_results.map(lambda ft: ft.set(
    'combined',
    ee.Number(ft.get('Corine')).gt(0).Or(ee.Number(ft.get('WorldCereal')).gt(1000))
))

# Filter features where 'combined' equals 1
loaded_collections = grid_results.filter(ee.Filter.eq('combined', 1))

# Function to filter borders for each country and combine them
def add_to_list(str, previous):
    previous = ee.List(previous)
    return previous.add(borders.filter(ee.Filter.eq('country_na', ee.String(str))).first())

# Accumulate borders for the specified countries
borders_ECA = ee.List(countries.iterate(add_to_list, ee.List([])))
borders_ECA = ee.FeatureCollection(borders_ECA)

# Filter loadedCollections to the bounds of the borders_ECA
loaded_collections = loaded_collections.filterBounds(borders_ECA)

# Map over loadedCollections to set 'fid' and 'scheme_property_name'
scheme_property_name = 'fid_property'  # Replace with your actual scheme property name
aoi_buffered = loaded_collections.map(lambda ft: ft.set(
    'fid', ee.Number(ft.get('fid'))
).set(
    scheme_property_name, ee.String('fid_').cat(ee.String(ft.get('fid')))
))

fids=aoi_buffered.aggregate_array('fid').getInfo()
print(len(fids))

# Get the first feature and its geometry
aoi_feature = aoi_buffered.filter(ee.Filter.eq('fid',27721)).first()
aoi_geometry = aoi_feature.geometry()

# Define the AOI
aoi = aoi_geometry

# Define a crop mask
WorldCereal=ee.ImageCollection("ESA/WorldCereal/2021/MODELS/v100");
CORINE=ee.Image("COPERNICUS/CORINE/V20/100m/2018")
CORINE_mosaic=CORINE.eq(212).Or(CORINE.eq(213)).selfMask()

# Define the LULC mask using WorldCereal and blend with CORINE mosaic
lulc = (
    WorldCereal.filter(ee.Filter.eq('product', 'temporarycrops')).max()
    .select('classification').gt(50).selfMask()
    .updateMask(
        WorldCereal.filter(ee.Filter.eq('product', 'temporarycrops')).max()
        .select('confidence').gt(50)
    )
    .unmask(0)
    .blend(CORINE_mosaic)
    .selfMask()
)

#keep only WorldCereal_temporarycrops, mask all the rest
#clip with country boundaries
#make sure the date of the WAPOR image is the middle of the month
#file name: use fid

490


In [7]:
first_year = 2021
last_year = 2021
wapor_et_data = load_wapor_et_data(
    first_year, last_year, frequency="monthly"
)#.filterBounds(aoi)


## Constants

In [161]:
#PATH_TO_AOI = "projects/thurgau-irrigation/assets/FribourgAndVaud/broye_bounding_box"
#PATH_TO_ET_PRODUCT = "projects/thurgau-irrigation/assets/ET_products/Thurgau/WaPOR_300m"
YEAR = 2021
USE_SR = False if YEAR < 2019 else True
BUFFER_DAYS = 15
BAND_TO_RESAMPLE = "ET"
#BANDS_TO_HARMONIZE = ["B3", "B4", "B8", "B11", "B12"]
BANDS_TO_HARMONIZE = ["NDVI", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "ST_B10"]
AGGREGATION_OPTIONS = {
    "agg_type": "mosaic",
    #"mosaic_type": "least_cloudy",
    "mosaic_type": "max_NDVI",
    "band_name": "NDVI",
}
INDEXES_FOR_HARMONIZATION = ["NDVI",  "NDWI","NDBI"]#,,"ST_B10"
INDEPENDENT_BANDS = ["gap_filled_NDVI", "gap_filled_NDWI", "gap_filled_NDBI"]#, "gap_filled_ST_B10"
DEPENDENT_BAND = ["ET"]
NUMBER_OF_IMAGES = 7
TEMPORAL_RESOLUTION = "monthly"
DOWNSCALED_ASSET_PATH = (
    f"projects/ee-et-data/assets/ECA/ET_data/WAPOR_downscaled_L8"
)

## 1. Load ET product

In [ ]:
#aoi_feature_collection = ee.FeatureCollection(PATH_TO_AOI)
#aoi_geometry = aoi_feature_collection.geometry().simplify(500)

#aoi = aoi_geometry.buffer(600)


# Map = geemap.Map()

# Map.centerObject(aoi, 10)
# Map.addLayer(aoi, {}, "AOI")


# Map

In [162]:
et_coarse_collection = (
    ee.ImageCollection(wapor_et_data)
    #ee.ImageCollection(PATH_TO_ET_PRODUCT)
    .filterBounds(aoi)
    .filterDate(f"{YEAR}-03-01", f"{YEAR}-10-31")
    .sort("system:time_start")
)

et_coarse_list = et_coarse_collection.toList(et_coarse_collection.size())

#print_collection_dates(et_coarse_collection)

## 2. Load Sentinel-2 data

In [150]:
s2_collection = load_sentinel2_data((YEAR, YEAR), aoi=aoi, use_SR=USE_SR)

time_intervals = create_centered_date_ranges(et_coarse_list, buffer_days=BUFFER_DAYS)
print(time_intervals.length().getInfo())
s2_harmonized = harmonized_ts(
    masked_collection=s2_collection,
    band_list=BANDS_TO_HARMONIZE,
    time_intervals=time_intervals,
    options=AGGREGATION_OPTIONS,
)

# s2_harmonized.first().bandNames().getInfo()

8


In [163]:
# Filter the Landsat 8 ImageCollection and apply the cloud masking function
l8_collection = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(aoi)
    .filter(ee.Filter.calendarRange(YEAR, YEAR, 'year'))
    .map(cloudscore_L8_T1L2)
)

time_intervals = create_monthly_date_ranges(et_coarse_list)

l8_harmonized = harmonized_ts(
    masked_collection=l8_collection,
    band_list=BANDS_TO_HARMONIZE,
    time_intervals=time_intervals,
    options=AGGREGATION_OPTIONS,
)
#print(l8_harmonized.size().getInfo())

#l8_harmonized.first().bandNames().getInfo()

### 2.1 Prepare the Sentinel-2 data for harmonization

In [164]:
def add_temporal_bands(collection: ee.ImageCollection) -> ee.ImageCollection:
    """Add temporal bands to each image in the collection."""
    def _add_bands(image: ee.Image) -> ee.Image:
        date = ee.Date(image.get('system:time_start'))
        years = date.difference(ee.Date('1970-01-01'), 'year')
        
        projection = image.select([0]).projection()
        time_band = ee.Image(years).float().rename('t')
        constant_band = ee.Image.constant(1).rename('constant')
        
        return image.addBands([
            time_band.setDefaultProjection(projection),
            constant_band.setDefaultProjection(projection)
        ])
    
    return collection.map(_add_bands)

#s2_harmonized = add_temporal_bands(s2_harmonized)
#s2_harmonized.first().bandNames().getInfo()

l8_harmonized = add_temporal_bands(l8_harmonized)
#l8_harmonized.first().bandNames().getInfo()

In [135]:
def compute_vegetation_indexes(image: ee.Image) -> ee.Image:
    """
    Compute vegetation indexes for a given image

    Args:
        image (ee.Image): The image to compute the vegetation indexes for

    Returns:
        ee.Image: The input image with the vegetation indexes

    """
    ndvi = image.normalizedDifference(["B8", "B4"]).rename("NDVI")
    ndwi = image.normalizedDifference(["B3", "B8"]).rename("NDWI")
    ndbi = image.normalizedDifference(["B11", "B8"]).rename("NDBI")
    return image.addBands(ndvi).addBands(ndwi).addBands(ndbi)

s2_harmonized_w_vegetation_indexes = s2_harmonized.map(compute_vegetation_indexes)

In [165]:
def compute_vegetation_indexes_landsat(image: ee.Image) -> ee.Image:
    """
    Compute vegetation indexes for Landsat 8 imagery.

    Args:
        image (ee.Image): The image to compute the vegetation indexes for.

    Returns:
        ee.Image: The input image with the vegetation indexes added.
    """
    ndvi = image.normalizedDifference(["SR_B5", "SR_B4"]).rename("NDVI")
    ndwi = image.normalizedDifference(["SR_B3", "SR_B5"]).rename("NDWI")
    ndbi = image.normalizedDifference(["SR_B6", "SR_B5"]).rename("NDBI")
    return image.addBands(ndvi).addBands(ndwi).addBands(ndbi)


# Compute vegetation indexes for the collection
l8_with_vegetation_indexes = l8_harmonized.map(compute_vegetation_indexes_landsat)

### 2.2 Fit an Harmonic Regressor to the Sentinel-2 data

In [166]:
#s2_harmonized_gaps_filled = s2_harmonized_w_vegetation_indexes
s2_harmonized_gaps_filled = l8_with_vegetation_indexes
#replace for loop by map
for index in INDEXES_FOR_HARMONIZATION:
    regressor = HarmonicRegressor(
        omega=1, max_harmonic_order=2, band_to_harmonize=index
    )

    #regressor.fit2(s2_harmonized_w_vegetation_indexes)
    regressor.fit2(l8_with_vegetation_indexes)
    #fitted_collection = regressor.predict(s2_harmonized_w_vegetation_indexes)
    fitted_collection = regressor.predict(l8_with_vegetation_indexes)

    fitted_collection = fitted_collection.map(
        lambda img: img.select(["fitted"]).rename(f"fitted_{index}")
    )

    s2_harmonized_gaps_filled = s2_harmonized_gaps_filled.map(
        lambda img: img.addBands(
            fitted_collection.filterDate(img.date()).first().select([f"fitted_{index}"])
        )
    )

In [ ]:
# s2_harmonized_gaps_filled.first().bandNames().getInfo()

### 2.3 Use the fitted harmonic bands to fill the gaps in the original bands

In [167]:
def fill_gaps(
    img: ee.Image, source_band: str, fill_band: str, output_name: str
) -> ee.Image:
    """Fill gaps in a band with values from another band.

    Args:
        img (ee.Image): Input image containing both bands
        source_band (str): Name of band containing gaps to fill
        fill_band (str): Name of band to use for filling gaps
        output_name (str): Name for the output gap-filled band

    Returns:
        ee.Image: Image with gap-filled band
    """
    scale = img.projection().nominalScale()
    projection = img.projection()
    # Create mask where the source band is invalid (gaps)
    gap_mask = img.select(source_band).mask().Not()

    # Get the source band and fill band
    source = img.select(source_band)
    fill = img.select(fill_band)

    # Fill gaps: use source band where available, fill band where there are gaps
    filled = source.unmask().where(gap_mask, fill).rename(output_name)
    filled = filled.setDefaultProjection(projection).set("scale", scale)

    return filled


def apply_gap_filling(img: ee.Image, indexes: List[str]) -> ee.Image:
    """Apply gap filling to multiple bands.

    Args:
        img (ee.Image): Input image
        indexes (list[str]): List of index names to process (e.g., ['NDVI', 'NDWI', 'NDBI'])

    Returns:
        ee.Image: Original image with added gap-filled bands
    """
    # Start with the original image
    result = img

    # Add each gap-filled band one at a time
    for index in indexes:
        filled_band = fill_gaps(
            img=img,
            source_band=index,
            fill_band=f"fitted_{index}",
            output_name=f"gap_filled_{index}",
        )
        result = result.addBands(filled_band)

    return result


# Apply gap filling to the collection
def process_collection(
    collection: ee.ImageCollection, indexes: List[str]
) -> ee.ImageCollection:
    """Process entire collection by applying gap filling to each image.

    Args:
        collection (ee.ImageCollection): Input collection
        indexes (List[str]): List of index names to process

    Returns:
        ee.ImageCollection: Processed collection with gap-filled bands
    """
    return collection.map(lambda img: apply_gap_filling(img, indexes))


s2_harmonized_gaps_filled = process_collection(s2_harmonized_gaps_filled, INDEXES_FOR_HARMONIZATION)

print(s2_harmonized_gaps_filled.first().bandNames().getInfo())

['NDVI', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'ST_B10', 't', 'constant', 'NDVI_1', 'NDWI', 'NDBI', 'fitted_NDVI', 'fitted_NDWI', 'fitted_NDBI', 'gap_filled_NDVI', 'gap_filled_NDWI', 'gap_filled_NDBI']


## 3. Downscale the ET product to 10m resolution

In [ ]:
def process_and_export_downscaled_ET(
    downscaler: Downscaler,
    s2_indices: ee.ImageCollection,
    independent_vars: ee.ImageCollection,
    dependent_vars: ee.ImageCollection,
    aoi: ee.Geometry,
    year: str,
    scale_coarse: float,
    asset_path: str,
    scale_fine: float = 10,
    time_steps: int = 36,
    time_step_type: str = "dekadal",
    fid_name: int = 1
) -> List[ee.batch.Task]:
    """
    Process and export downscaled WaPOR ET images to Earth Engine assets.

    Args:
        downscaler (Downscaler): The Downscaler object used to downscale the images.
        s2_indices (ee.ImageCollection): The Sentinel-2 indices image collection.
        independent_vars (ee.ImageCollection): The resampled independent variables image collection.
        dependent_vars (ee.ImageCollection): The dependent variables image collection.
        aoi (ee.Geometry): The area of interest geometry.
        year (str): The year for which the images are processed.
        scale_coarse (float): The scale of the images before downscaling.
        scale_fine (float): The scale of the images after downscaling.
        time_steps (int): Number of time steps in the year (36 for dekadal, 12 for monthly).
        time_step_type (str): Type of time step ("dekadal" or "monthly").
        fid_name (int): fid of image

    Returns:
        List[ee.batch.Task]: A list of export tasks for the downscaled images.
    """
    s2_indices_list = s2_indices.toList(s2_indices.size()).slice(1)
    independent_vars_list = independent_vars.toList(independent_vars.size()).slice(1)
    dependent_vars_list = dependent_vars.toList(dependent_vars.size()).slice(1)

    tasks = []
    for i in range(time_steps):
        if time_step_type == "dekadal":
            j = i % 3 + 1
            m = i // 3 + 1
            date = ee.Date.fromYMD(int(year), m, j * 10 - 9)
            time_step_name = f"{m:02d}_D{j}"
        elif time_step_type == "monthly":
            m = i + 4
            date = ee.Date.fromYMD(int(year), m, 1)
            time_step_name = f"{m:02d}"
        else:
            raise ValueError("time_step_type must be either 'dekadal' or 'monthly'")

        s2_index = ee.Image(s2_indices_list.get(i))
        ind_vars = ee.Image(independent_vars_list.get(i))
        dep_vars = ee.Image(dependent_vars_list.get(i))

        # Perform downscaling
        et_image_downscaled = downscaler.downscale(
            coarse_independent_vars=ind_vars,
            coarse_dependent_var=dep_vars,
            fine_independent_vars=s2_index,
            geometry=aoi,
            resolution=scale_coarse,
        )

        # Post-process the downscaled image
        et_image_downscaled = back_to_int(et_image_downscaled, 100)
       
        et_image_downscaled = et_image_downscaled.set('Year',int(year)).set('Month',m).set('fid',fid_name)
        #task_name = f"Downscaled_ET_gap_filled_{time_step_type}_{year}_{time_step_name}_F{fid_name}"
        task_name = f"Downscaled_ET_WAPOR_L8_{time_step_type}_{year}_{time_step_name}_F{fid_name}"
        asset_id = f"{asset_path}/{task_name}"

        task = export_image_to_asset(
            et_image_downscaled.max(ee.Image(0)).clip(borders_ECA.geometry()),#.updateMask(lulc.eq(1))
            asset_id,
            task_name,
            year,
            aoi,
            #crs="EPSG:32632",
            scale=scale_fine,
        )
        tasks.append(task)

In [169]:
scale = et_coarse_collection.first().projection().nominalScale().getInfo()

s2_indices = s2_harmonized_gaps_filled.select(INDEPENDENT_BANDS)
independent_vars = s2_indices.map(
    lambda img: resample_image(img, scale, INDEPENDENT_BANDS)
)

dependent_vars = et_coarse_collection.select(DEPENDENT_BAND)


# Initialize the Downscaler
downscaler = Downscaler(
    independent_bands=INDEPENDENT_BANDS,  
    dependent_band=DEPENDENT_BAND[0],
)


tasks = process_and_export_downscaled_ET(
    downscaler=downscaler,
    s2_indices=s2_indices,
    independent_vars=independent_vars,  
    dependent_vars=dependent_vars,
    aoi=aoi,
    year=YEAR,
    scale_coarse=scale,
    scale_fine=30,
    time_steps=NUMBER_OF_IMAGES,
    time_step_type=TEMPORAL_RESOLUTION,
    asset_path=DOWNSCALED_ASSET_PATH,
    fid_name=27721,
)

Exporting Downscaled_ET_WAPOR_L8_monthly_2021_04_F27721 for 2021 to projects/ee-et-data/assets/ECA/ET_data/WAPOR_downscaled_L8/Downscaled_ET_WAPOR_L8_monthly_2021_04_F27721
Exporting Downscaled_ET_WAPOR_L8_monthly_2021_05_F27721 for 2021 to projects/ee-et-data/assets/ECA/ET_data/WAPOR_downscaled_L8/Downscaled_ET_WAPOR_L8_monthly_2021_05_F27721
Exporting Downscaled_ET_WAPOR_L8_monthly_2021_06_F27721 for 2021 to projects/ee-et-data/assets/ECA/ET_data/WAPOR_downscaled_L8/Downscaled_ET_WAPOR_L8_monthly_2021_06_F27721
Exporting Downscaled_ET_WAPOR_L8_monthly_2021_07_F27721 for 2021 to projects/ee-et-data/assets/ECA/ET_data/WAPOR_downscaled_L8/Downscaled_ET_WAPOR_L8_monthly_2021_07_F27721
Exporting Downscaled_ET_WAPOR_L8_monthly_2021_08_F27721 for 2021 to projects/ee-et-data/assets/ECA/ET_data/WAPOR_downscaled_L8/Downscaled_ET_WAPOR_L8_monthly_2021_08_F27721
Exporting Downscaled_ET_WAPOR_L8_monthly_2021_09_F27721 for 2021 to projects/ee-et-data/assets/ECA/ET_data/WAPOR_downscaled_L8/Downscal

In [ ]:
independent_vars.first().bandNames().getInfo()

In [158]:
print(len(fids[432:]))
index = fids.index(27721)  # Get the index of 27300
print(index)  # Output: 2

58
4


In [ ]:
##Batch task

for fid in fids[250:]:
    # Filter the aoi_buffered collection for the current fid
    aoi_fid = aoi_buffered.filter(ee.Filter.eq('fid', fid)).first().geometry()
    
    et_coarse_collection = (
        ee.ImageCollection(wapor_et_data)
        .filterBounds(aoi_fid)
        .filterDate(f"{YEAR}-03-01", f"{YEAR}-10-31")
        .sort("system:time_start")
    )
    et_coarse_list = et_coarse_collection.toList(et_coarse_collection.size())

    #s2_collection = load_sentinel2_data((YEAR, YEAR), aoi=aoi_fid, use_SR=USE_SR)
    l8_collection = (
        ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
        .filterBounds(aoi_fid)
        .filter(ee.Filter.calendarRange(YEAR, YEAR, 'year'))
        .map(cloudscore_L8_T1L2)
    )
    time_intervals = create_monthly_date_ranges(et_coarse_list)

    l8_harmonized = harmonized_ts(
        masked_collection=l8_collection,
        band_list=BANDS_TO_HARMONIZE,
        time_intervals=time_intervals,
        options=AGGREGATION_OPTIONS,
    )
    l8_harmonized = add_temporal_bands(l8_harmonized)
    s2_harmonized_w_vegetation_indexes = l8_harmonized.map(compute_vegetation_indexes_landsat)

    l8_with_vegetation_indexes = s2_harmonized_w_vegetation_indexes
    s2_harmonized_gaps_filled = l8_with_vegetation_indexes

    for index in INDEXES_FOR_HARMONIZATION:
        regressor = HarmonicRegressor(
            omega=1, max_harmonic_order=2, band_to_harmonize=index
        )
        regressor.fit2(l8_with_vegetation_indexes)
        fitted_collection = regressor.predict(l8_with_vegetation_indexes)

        fitted_collection = fitted_collection.map(
            lambda img: img.select(["fitted"]).rename(f"fitted_{index}")
        )

        s2_harmonized_gaps_filled = s2_harmonized_gaps_filled.map(
            lambda img: img.addBands(
                fitted_collection.filterDate(img.date()).first().select([f"fitted_{index}"])
            )
        )

    s2_harmonized_gaps_filled = process_collection(s2_harmonized_gaps_filled, INDEXES_FOR_HARMONIZATION)

    s2_indices = s2_harmonized_gaps_filled.select(INDEPENDENT_BANDS)
    independent_vars = s2_indices.map(
        lambda img: resample_image(img, scale, INDEPENDENT_BANDS)
    )

    dependent_vars = et_coarse_collection.select(DEPENDENT_BAND)

    # Initialize the Downscaler
    downscaler = Downscaler(
        independent_bands=INDEPENDENT_BANDS,  
        dependent_band=DEPENDENT_BAND[0],
    )

    tasks = process_and_export_downscaled_ET(
        downscaler=downscaler,
        s2_indices=s2_indices,
        independent_vars=independent_vars,  
        dependent_vars=dependent_vars,
        aoi=aoi_fid,
        year=YEAR,
        scale_coarse=scale,
        scale_fine=30,
        time_steps=NUMBER_OF_IMAGES,
        time_step_type=TEMPORAL_RESOLUTION,
        asset_path=DOWNSCALED_ASSET_PATH,
        fid_name=fid,
    )
    # Call the export_batch_task function with the appropriate arguments
    #export_batch_task(asset_path, filename, fid, aoi_fid)


In [172]:
assetdir = f"projects/ee-et-data/assets/{'ECA/ET_data/WAPOR_downscaled_L8'}"
assets = ee.data.listAssets({"parent": assetdir})["assets"]

for table in assets:
    id = table['id']
    ee.data.deleteAsset(id)
    print('Deleted asset ID: ' + id)

Deleted asset ID: projects/ee-et-data/assets/ECA/ET_data/WAPOR_downscaled_L8/Downscaled_ET_WAPOR_L8_monthly_2021_04_F23389
Deleted asset ID: projects/ee-et-data/assets/ECA/ET_data/WAPOR_downscaled_L8/Downscaled_ET_WAPOR_L8_monthly_2021_04_F27300
Deleted asset ID: projects/ee-et-data/assets/ECA/ET_data/WAPOR_downscaled_L8/Downscaled_ET_WAPOR_L8_monthly_2021_04_F27301
Deleted asset ID: projects/ee-et-data/assets/ECA/ET_data/WAPOR_downscaled_L8/Downscaled_ET_WAPOR_L8_monthly_2021_04_F27712
Deleted asset ID: projects/ee-et-data/assets/ECA/ET_data/WAPOR_downscaled_L8/Downscaled_ET_WAPOR_L8_monthly_2021_04_F27713
Deleted asset ID: projects/ee-et-data/assets/ECA/ET_data/WAPOR_downscaled_L8/Downscaled_ET_WAPOR_L8_monthly_2021_04_F27721
Deleted asset ID: projects/ee-et-data/assets/ECA/ET_data/WAPOR_downscaled_L8/Downscaled_ET_WAPOR_L8_monthly_2021_04_F28127
Deleted asset ID: projects/ee-et-data/assets/ECA/ET_data/WAPOR_downscaled_L8/Downscaled_ET_WAPOR_L8_monthly_2021_05_F23389
Deleted asset ID

## Sanity checks
### 1. Check the Sentinel-2 harmonization

In [91]:
Map = geemap.Map()

sentinel_vis_params = {
    "bands": ["NDVI"],
    "min": 0,
    "max": 1,
    "palette": ["white", "green"],
}

fitted_vis_params = {
    "bands": ["fitted_NDVI"],
    "min": 0,
    "max": 1,
    "palette": ["white", "green"],
}

gap_filled_vis_params = {
    "bands": ["gap_filled_NDVI"],
    "min": 0,
    "max": 1,
    "palette": ["white", "green"],
}


s2_image = ee.Image(s2_harmonized_gaps_filled.filterBounds(aoi).toList(36).get(-3))
s2_image_resampled = ee.Image(independent_vars.toList(36).get(0))

Map.addLayer(s2_image, sentinel_vis_params, "Sentinel 2")
# # Map.addLayer(s2_image, fitted_vis_params, "Sentinel 2 Fitted")
Map.addLayer(s2_image, gap_filled_vis_params, "Sentinel 2 Gap Filled")
# Map.addLayer(s2_image_resampled, gap_filled_vis_params, "Sentinel 2 Resampled")


Map.centerObject(aoi, 12)
Map

Map(center=[47.749693482876474, 22.250000107056604], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# print_collection_dates(s2_harmonized_gaps_filled)

In [108]:
Map = geemap.Map()

sentinel_vis_params = {
    "bands": ["ST_B10"],
    "min": 304-273,
    "max": 320-273,
    "palette": ["white", "green"],
}

fitted_vis_params = {
    "bands": ["fitted_ST_B10"],
    "min": 304-273,
    "max": 300-273,
    "palette": ["white", "green"],
}

gap_filled_vis_params = {
    "bands": ["gap_filled_ST_B10"],
    "min": 304-273,
    "max": 320-273,
    "palette": ["white", "green"],
}


s2_image = ee.Image(s2_harmonized_gaps_filled.filterBounds(aoi).toList(36).get(-3))
s2_image_resampled = ee.Image(independent_vars.toList(36).get(0))

Map.addLayer(s2_image, sentinel_vis_params, "Sentinel 2")
Map.addLayer(s2_image, fitted_vis_params, "Sentinel 2 Fitted")
#Map.addLayer(s2_image, gap_filled_vis_params, "Sentinel 2 Gap Filled")
# Map.addLayer(s2_image_resampled, gap_filled_vis_params, "Sentinel 2 Resampled")


Map.centerObject(aoi, 12)
Map

Map(center=[47.749693482876474, 22.250000107056604], controls=(WidgetControl(options=['position', 'transparent…

TraitError: setting lower > upper

### 2. Compute the ET downscaled images

In [ ]:
# collection = ee.ImageCollection("projects/thurgau-irrigation/assets/ET_products/Thurgau/WaPOR_10m_2018").map(lambda img: back_to_float(img, 100))

# print_collection_dates(collection)

In [ ]:
# collection_list = collection.toList(collection.size())


In [ ]:
# Map = geemap.Map()

# et_vis_params = {
#     "bands": ["downscaled"],
#     "min": 0,
#     "max": 5,
#     "palette": ["white", "lightblue", "blue", "green", "yellow", "orange", "red", "darkred"],
# }

# et_image = ee.Image(collection_list.get(14))

# Map.addLayer(et_image, et_vis_params, "ET")

# Map.centerObject(aoi, 12)

# Map

In [ ]:
# sentinel_collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(aoi).filterDate("2019-5-01", "2019-05-31").sort("system:time_start")

# # Get list of unique dates and merge same-day images
# def merge_daily_images(collection):
#     # Get list of unique dates
#     dates = collection.aggregate_array('system:time_start').distinct()
    
#     def merge_images_for_date(date):
#         # Get images for this date
#         daily_images = collection.filter(ee.Filter.eq('system:time_start', date))
#         # Merge images (mosaic takes the first non-masked pixel)
#         merged = daily_images.mosaic()
#         # Set the date property
#         return merged.set('system:time_start', date)
    
#     # Map merge function over dates
#     merged_collection = ee.ImageCollection(dates.map(lambda date: merge_images_for_date(date)))
#     return merged_collection

# # Apply the merging
# merged_sentinel = merge_daily_images(sentinel_collection)

# # Visualize first merged image
# merged_image = ee.Image(merged_sentinel.first())

# # Print number of images before and after
# print(f"Original collection size: {sentinel_collection.size().getInfo()}")
# print(f"Merged collection size: {merged_sentinel.size().getInfo()}")

In [ ]:
# Map = geemap.Map()

# sentinel_vis_params = {
#     "bands": ["B4", "B3", "B2"],
#     "min": 0,
#     "max": 2000,
# }

# sentinel_image = ee.Image(sentinel_collection.first())

# Map.addLayer(sentinel_image, sentinel_vis_params, "Sentinel 2")
# Map.addLayer(merged_image, sentinel_vis_params, "Merged Sentinel 2")

# Map.centerObject(aoi, 12)

# Map